In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import cv2
import time
import argparse
import posenet

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
parser = argparse.ArgumentParser()

In [4]:
parser.add_argument('--model', type=int, default=101)
parser.add_argument('--cam_id', type=int, default=0)

_StoreAction(option_strings=['--cam_id'], dest='cam_id', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help=None, metavar=None)

In [5]:
parser.add_argument('--cam_width', type=int, default=1280)
parser.add_argument('--cam_height', type=int, default=720)
parser.add_argument('--scale_factor', type=float, default=0.7125)

_StoreAction(option_strings=['--scale_factor'], dest='scale_factor', nargs=None, const=None, default=0.7125, type=<class 'float'>, choices=None, help=None, metavar=None)

In [6]:
parser.add_argument('--file', type=str, default=None, help="Optionally use a video file instead of a live camera")

_StoreAction(option_strings=['--file'], dest='file', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='Optionally use a video file instead of a live camera', metavar=None)

In [7]:
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--model MODEL] [--cam_id CAM_ID] [--cam_width CAM_WIDTH] [--cam_height CAM_HEIGHT]
                             [--scale_factor SCALE_FACTOR] [--file FILE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\COM\AppData\Roaming\jupyter\runtime\kernel-c76b4d03-a3d5-42ec-946f-eab5e389f895.json


SystemExit: 2

C:\Users\COM\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
def main():
    with tf.Session() as sess:
        model_cfg, model_outputs = posenet.load_model(args.model, sess)
        output_stride = model_cfg['output_stride']

        if args.file is not None:
            cap = cv2.VideoCapture(args.file)
        else:
            cap = cv2.VideoCapture(args.cam_id)
        cap.set(3, args.cam_width)
        cap.set(4, args.cam_height)

        start = time.time()
        frame_count = 0
        while True:
            input_image, display_image, output_scale = posenet.read_cap(
                cap, scale_factor=args.scale_factor, output_stride=output_stride)

            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                model_outputs,
                feed_dict={'image:0': input_image}
            )

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                heatmaps_result.squeeze(axis=0),
                offsets_result.squeeze(axis=0),
                displacement_fwd_result.squeeze(axis=0),
                displacement_bwd_result.squeeze(axis=0),
                output_stride=output_stride,
                max_pose_detections=10,
                min_pose_score=0.15)

            keypoint_coords *= output_scale

            # TODO this isn't particularly fast, use GL for drawing and display someday...
            overlay_image = posenet.draw_skel_and_kp(
                display_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0.15, min_part_score=0.1)

            cv2.imshow('posenet', overlay_image)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        print('Average FPS: ', frame_count / (time.time() - start))

In [11]:
if __name__ == "__main__":
    main()

NameError: name 'args' is not defined

NameError: name 'args' is not defined